In [ ]:
#Install FLAML
!pip install flaml

In [ ]:
#Import Libraries

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from flaml import AutoML
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Memory Reduction Technique

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                    
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
df_train_ogi = import_data("../input/tabular-playground-series-oct-2021/train.csv")
df_test_ogi = import_data("../input/tabular-playground-series-oct-2021/test.csv")   

In [ ]:
X = df_train_ogi.drop(['id','target'], axis = 1)
y = df_train_ogi['target']

In [ ]:
sample = import_data("../input/tabular-playground-series-oct-2021/sample_submission.csv")   

In [ ]:
automl = AutoML()
automlfit = automl.fit(X_train = X, y_train = y, metric = 'roc_auc', time_budget = 3600)

In [ ]:
sample['target'] = automl.predict_proba(df_test_ogi)

In [ ]:
sample.to_csv('submission.csv', index = False)

In [ ]:
sample.head(20)